## Bibliothèques nécessaire pour la simulation des agents

In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter                                               

## Bibliothèques intermédiares

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle

### Approximation des équations différentielles en utilisant EULEUR

In [3]:
def Euler(So, Io, Ro, l, b, u):
    N = 20
    T = 1.0
    h = T/N
    
    t = [h * i for i in range(N+1)]
    Sn, In, Rn = So, Io, Ro
    
    for to in t[1:]:
        Sn_1 =  Sn - h*b*In*Sn
        In_1 = In + h*(b*In*Sn - (In/l) -u*In)
        Rn_1 = Rn + h*(In/l)
        Sn, In, Rn = Sn_1, In_1, Rn_1
        
    return Sn, In, Rn

## Modélisation des agents et Simulation du virus

In [4]:
class PersonAgent(Agent):
        
    def __init__(self, unique_id, agent_type, model):
        
        self.agent_type = agent_type
        
        super().__init__(unique_id, model)
        
    def move(self):
        x, y = self.pos
        walk = [(0, 0), (0, 1), (0, -1), (1, 0), (1, 1), (1, -1), (-1, -1), (-1, 1), (-1, 0) ]
        xd, yd = walk[np.random.choice(range(len(walk)))]
        self.model.grid.move_agent(self, (x+xd, y+yd))
        
    def step(self):    
        self.move()

In [5]:
class SIR(Model):
    def __init__(self, n_agents, width, height, taux_infecte, taux_transmission, temps_guerison, taux_mortalite):
        super().__init__()
        
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, torus=True)
        self.last_id = 1
        self.n_agents = n_agents
        
        # Paramètres du modèle
        self.taux_transmission, self.temps_guerison, self.taux_mortalite = taux_transmission, temps_guerison, taux_mortalite
        self.I = taux_infecte
        self.S = 1 - self.I
        self.R = 0
        
        n_I = round(self.I * n_agents)
        n_S = round(self.S * n_agents)
        n_R = round(self.R * n_agents)
        
        # Tracks des données
        self.sains = n_S
        self.infectes = n_I
        self.retablis = n_R
        self.population = n_S + n_I + n_R
        self.isFistTime = True
        
        for _ in range(n_S):
            a = PersonAgent(self.last_id, "S", self)
            self.schedule.add(a)
            coords = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            self.grid.place_agent(a, coords)
            self.last_id += 1
        
        for _ in range(n_I):
            a = PersonAgent(self.last_id, "I", self)
            self.schedule.add(a)
            coords = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            self.grid.place_agent(a, coords)
            self.last_id += 1
        
        self.datacollector = DataCollector({
            'Population': 'nb_population',
            'Sains': 'nb_sains',
            'Infectés': 'nb_infectes',
            'Rétablis': 'nb_retablis'})
        
        self.datacollector.collect(self)
    
    @property
    def nb_population(self):
        return self.population
    
    @property
    def nb_sains(self):
        return self.sains
    
    @property
    def nb_infectes(self):
        return self.infectes
    
    @property
    def nb_retablis(self):
        return self.retablis

    
    
    def step(self):
        
        #steps = self.schedule.steps
        S, I, R = Euler(self.S, self.I, self.R, self.temps_guerison, self.taux_transmission, self.taux_mortalite)
        
        n_I = round(I * self.n_agents)
        n_S = round(S * self.n_agents)
        n_R = round(R * self.n_agents)
        
        n_ecart_I = n_I - self.infectes
        n_ecart_S = n_S - self.sains
        n_ecart_R = n_R - self.retablis
        
        
        list_agents = self.schedule.agents.copy()
        shuffle(list_agents)
        
        for agent in list_agents:
            
            if self.isFistTime == True:
                if (n_ecart_R) > 0:
                    a = PersonAgent(self.last_id, "R", self)
                    self.schedule.add(a)
                    coords = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
                    self.grid.place_agent(a, coords)
                    self.last_id += 1
                    n_ecart_R -= 1
                self.isFistTime = False
                
            if (n_ecart_S == 0) and (n_ecart_I == 0) and (n_ecart_R == 0):
                break
                
            if agent.agent_type == "I":
                # Ajout de I
                if (n_ecart_I) > 0:
                    a = PersonAgent(self.last_id, "I", self)
                    self.schedule.add(a)
                    coords = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
                    self.grid.place_agent(a, coords)
                    self.last_id += 1
                    n_ecart_I -= 1
                    
                # Suppression de I
                if (n_ecart_I) < 0:
                    self.grid.remove_agent(agent)
                    self.schedule.remove(agent)
                    n_ecart_I += 1
            
            if agent.agent_type == "S":
                # Ajout de S
                if (n_ecart_S) > 0:
                    a = PersonAgent(self.last_id, "S", self)
                    self.schedule.add(a)
                    coords = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
                    self.grid.place_agent(a, coords)
                    self.last_id += 1
                    n_ecart_S -= 1
                    
                if (n_ecart_S) < 0:
                    self.grid.remove_agent(agent)
                    self.schedule.remove(agent)
                    n_ecart_S += 1
                    
            if agent.agent_type == "R":
                
                # Ajout de R
                if (n_ecart_R) > 0:
                    
                    a = PersonAgent(self.last_id, "R", self)
                    self.schedule.add(a)
                    coords = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
                    self.grid.place_agent(a, coords)
                    self.last_id += 1
                    n_ecart_R -= 1
                    
                if (n_ecart_R) < 0:
                    self.grid.remove_agent(agent)
                    self.schedule.remove(agent)
                    n_ecart_R += 1
                
                
        # Tracks ou sauvegarde des données
        self.sains, self.infectes, self.retablis = n_S, n_I, n_R
        self.population = n_S + n_I + n_R
        
        self.S, self.I, self.R = S, I, R
        
        self.datacollector.collect(self)
        self.schedule.step()

In [6]:
model_params = {
    'n_agents': UserSettableParameter(
        'slider', 'Population (Nb Agent)', 1000, 100, 5000, 5),
    
    'width': 50,
    'height': 50,
    'taux_infecte': UserSettableParameter(
        'slider', 'Individus infectés initial(%)', 0.2, 0, 1, 0.01),
    
    'taux_transmission': UserSettableParameter(
        'slider', 'Taux de transmission (%)', 0.5, 0, 1, 0.01),
    
    'temps_guerison': UserSettableParameter(
        'slider', 'Temps de guérison (jour)', 12, 1, 30, 1),
    
    'taux_mortalite': UserSettableParameter(
        'slider', 'Taux de mortalité (%)', 0.05, 0, 1, 0.01),
}

In [7]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 1,
                }
    
    if agent.agent_type == "I":
        portrayal["Color"] = "red"
        
    if agent.agent_type == "R":
        portrayal["Color"] = "green"
        
    if agent.agent_type == "S":
        portrayal["Color"] = "black"
        
        
        
    return portrayal

grid = CanvasGrid(agent_portrayal, 50, 50, 550, 500)

line_charts = ChartModule([
    {'Label': 'Population', 'Color': 'lightblue'}, 
    {'Label': 'Sains', 'Color': 'black'},
    {'Label': 'Infectés', 'Color': 'red'},
    {'Label': 'Rétablis', 'Color': 'green'}])

server = ModularServer(SIR,
                       [grid, line_charts],
                       'Simulation de Propagation du virus',
                       model_params)

server.port = 8085
server.launch()

Interface starting at http://127.0.0.1:8085


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}


{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step","step":33}
{"type":"get_step",

{"type":"get_step","step":56}
{"type":"get_step","step":57}
{"type":"get_step","step":58}
{"type":"get_step","step":59}
{"type":"get_step","step":60}
{"type":"get_step","step":61}
{"type":"get_step","step":62}
{"type":"get_step","step":63}
{"type":"get_step","step":64}
{"type":"get_step","step":65}
{"type":"get_step","step":66}
{"type":"get_step","step":67}
{"type":"get_step","step":68}
{"type":"get_step","step":69}
{"type":"get_step","step":70}
{"type":"get_step","step":71}
{"type":"get_step","step":72}
{"type":"get_step","step":73}
{"type":"submit_params","param":"taux_mortalite","value":0.19}
{"type":"submit_params","param":"taux_mortalite","value":0.18}
{"type":"submit_params","param":"taux_mortalite","value":0.17}
{"type":"submit_params","param":"taux_mortalite","value":0.16}
{"type":"submit_params","param":"taux_mortalite","value":0.15}
{"type":"submit_params","param":"taux_mortalite","value":0.14}
{"type":"submit_params","param":"taux_mortalite","value":0.13}
{"type":"submit_par